**Requirments**

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install emoji
!pip install unidecode


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.4 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 6.6 MB 40.4 MB/s 
     |████████████████████████████████| 596 kB 30.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 6.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=f3dd068135f33eaa3253ea839c7b9c94749fd42bd4640478a09a90d14b099b34
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01d

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import torch
import os
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm 
import torch.nn.functional as F


**config**

In [ ]:
EPOCHS = 4
#DATA_PATH ="/content/drive/MyDrive/Colab_Notebooks/classification_camembert/class_data1.csv"
model_name='camembert-base'

**Utils**

In [ ]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {'accuracy': acc,}

**Cleaning**

In [ ]:
import random
import pandas as pd
env_label=['écologie', 'énergie', 'climat', 'pollution', 'environnement', 'gaz', 'nature', "recyclage", 'déforestation', 'urbanisation', 'ressources naturelles', 'biologique', 'poubelle', 'nucléaire',  'forage',  'pesticide', 'renouvelable',  'éboueur', 'déchet', 'ordures', 'contamination']

imm_label=['migration', 'immigration', 'asile', 'réfugié', 'immigré', 'étranger', 'clandestin', 'visa', 'émigration', 'nationalité', 'colon', 'exilé', 'migratoire', 'frontières', 'exode', 'transhumance', 'délocalisation', 'déménager', 'déplacement', 'invasion', 'fuite', 'escapade', 'exil', 'exode', 'nomade', 'voyageur', 'fuite des cerveaux', 'intégration', 'regroupement familial', 'fuyant']

eco_label= ["pouvoir achat", 'taxe','coût' ,'inflation', 'consommation', 'prix',"fiscalité",'salaire','revenus', 'TVA', 'impôts', 'retraite','smic','argent', 'crédit','économie','épargne','loyers','dette','Taux d’intérêt' , 'nettes', 'marchandises' ,"financer", "augmentation des salaires" , "richesse", "rémunération","contrat","achat","finance", 'vente','marché', 'cotisation']

sante_label=['santé', 'virus', 'maladie', 'médecine', 'médecin','vaccin', 'covid', 'masque', 'hygiène', 'bactérie', 'médicament', 'hôpital',  'épidémie', 'assurance', 'prévention sanitaire', 'remède', 'malade', 'patient', 'thérapie', 'handicap', 'clinique', 'pharmacie', 'pharmacien', 'guérison', 'soin', 'vacciné', 'infirmier']

sec_label=['sécurité', 'insécurité', 'agression', 'défense', 'délinquance', 'terrorisme', 'police', 'violence', 'criminalité', 'paix', 'surveillance', 'protection', 'guerre', 'soldat', 'menace', 'danger', 'piratage', 'cybersécurité', 'attaque', 'gendarmerie', 'armée']

ret_label=['retraite', 'préretraite', 'pension de retraite', 'fonds de pension', "caisses d'épargne" , 'les vieux', 'assurance-retraite', '65 ans', '60 ans' , 'âge de départ', 'retraité', 'vieillesse']

egal_label=['inégalité', 'racisme', 'discrimination', 'justice', 'équité', 'égalitaire', 'hiérarchie', 'démocratique', 'caste', 'sexisme', 'préjugé', 'préférence', 'persécution', 'déséquilibre', 'disparité', 'disproportion', 'dissemblance', 'ségrégation', 'inégalitarisme', 'droits', 'misogynie', 'fanatisme', 'chauvinisme', 'xénophobie', 'incitation', 'humiliation', 'insulte', 'intolérance', 'supériorité', 'esclavage']

laicite_label=['juif','chritien','laïcité', 'laïque', 'religion d’état', 'religieux', 'laïcisation', 'sécularité', 'séparation', 'islamophobie', 'religion','signe religieux', 'islamofascistes', 'voile' , 'séparatisme religieux' , 'loi de 1905' , 'musulman']
df = pd.DataFrame(columns = ['text', 'label'])
for i in range(500):
  df = df.append({'text': " ".join(random.choices(eco_label, k=20)), 'label': 0}, ignore_index=True)
for i in range(500):
  df = df.append({'text': " ".join(random.choices(env_label, k=20)), 'label': 1}, ignore_index=True)
for i in range(500):
  df = df.append({'text': " ".join(random.choices(imm_label, k=20)), 'label': 2}, ignore_index=True)
for i in range(500):
  df = df.append({'text': " ".join(random.choices(egal_label, k=20)), 'label': 3}, ignore_index=True)
for i in range(500):
  df = df.append({'text': " ".join(random.choices(laicite_label, k=20)), 'label': 4}, ignore_index=True)
for i in range(500):
  df = df.append({'text': " ".join(random.choices(ret_label, k=20)), 'label': 5}, ignore_index=True)
for i in range(500):
  df = df.append({'text': " ".join(random.choices(sante_label, k=20)), 'label': 6}, ignore_index=True)
for i in range(500):
  df = df.append({'text': " ".join(random.choices(sec_label, k=20)), 'label': 7}, ignore_index=True)
df.to_csv("/content/drive/MyDrive/Colab_Notebooks/classification_camembert/mots_cle500.csv", index=False)

In [ ]:
from emoji import UNICODE_EMOJI
import unidecode
import re

class clean_data():
    # remove URLs   
    def remove_URL(self,text):
        url = re.compile(r'https?://\S+|www\.\S+')
        return url.sub(r'',text)

    # remove htmls
    def remove_html(self,text):
        html=re.compile(r'<.*?>')
        return html.sub(r'',text)
    
    # remove punct
    def remove_punct(self,text):
        punc = '''!()[]{};:"\,<>./?@$%^&#*_~'''
        for ele in text:
            if ele in punc:
                text= text.replace(ele, "")
        return text
    # remove other ...
    def remove_other (self,text) : 
        text = text.rstrip()
        text = re.sub(' +', ' ', text)
        text = re.sub('-', ' ', text)
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('\n', '', text)
        text = re.sub('\w*\d\w*', '', text)
        return(text)

    # lower   
    def lower_text(self, text):
        return text.lower()

    # remove emojis
    def remove_emoji(self, text):
        to_replace = UNICODE_EMOJI["en"]
        result = text
        for x in to_replace:
            result = result.replace(x, "")
        return result

    # remove accents
    def remove_accents(self, text):
        return unidecode.unidecode(text)

    # clean text
    def clean(self,text) : 
        text = self.lower_text(text)
        text = self.remove_URL(text)
        text = self.remove_html(text)
        text = self.remove_punct(text)
        text = self.remove_emoji(text)
        text = self.remove_other (text)
        return text


**Dataset**

In [ ]:
import torch
from torch.utils.data import Dataset , DataLoader
class Dataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings= encodings
        self.labels= labels
    
    def __getitem__(self, idx):
        item={key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels']=torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)


**Fine Tunning**

In [ ]:

def fine_tuning(train_texts,test_texts,train_label,test_label):
  print('Step 1 : Loading Model & Tokenizer ')
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7)
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  print('Step 2 : Encoding Tokens')
  train_encodings= tokenizer(train_texts,truncation=True,padding=True,max_length = 30)
  test_encoding=tokenizer(test_texts,truncation=True,padding=True,max_length = 30)
  print ('Step 3 : Creating Dataset')
  traindt = Dataset(train_encodings,train_label)
  testdt=Dataset(test_encoding,test_label)  
  training_args = TrainingArguments(
      output_dir="/content/drive/MyDrive/Colab_Notebooks/classification_camembert/log/",
      num_train_epochs=1,
      per_device_train_batch_size=64,
      per_device_eval_batch_size=64,
      warmup_steps=500,
      weight_decay=0.01,
      evaluation_strategy='epoch',
      logging_dir='/content/drive/MyDrive/Colab_Notebooks/classification_camembert/log/',
      learning_rate= 0.00005,
      logging_steps=80000,
      save_strategy="no"
  )
  print ('Step 4 : Training ')
  for i in tqdm(range(1,6)) : 
    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=traindt,
        eval_dataset=testdt
        )
    trainer.train()
    PATH_TO_SAVE = f"/content/drive/MyDrive/Colab_Notebooks/classification_camembert/epochs/Epoch_{i}"
    model.save_pretrained(PATH_TO_SAVE)
    model = AutoModelForSequenceClassification.from_pretrained(PATH_TO_SAVE)
  return traindt , testdt


In [ ]:
"""
#"/content/drive/MyDrive/Colab_Notebooks/classification_camembert/train_cles.csv"
train= pd.read_csv("/content/drive/MyDrive/train.csv")
train.dropna(subset=['text'],inplace=True)
X_train= (train.text.values.tolist())
y_train=train.label.values.tolist()
#train_texts,test_texts,train_label,test_label= train_test_split(comments,labels,shuffle=True,test_size=0.2)
test= pd.read_csv("/content/drive/MyDrive/test.csv")
test.dropna(subset=['text'],inplace=True)
X_test= (test.text.values.tolist())
y_test=test.label.values.tolist()"""

#data= pd.read_csv("/content/drive/MyDrive/Full_Data.csv")
#data=data[data['label']==6]
cles=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/classification_camembert/mots_cle500.csv")
"""
cles=cles[cles['label']!=5]
cles=cles[cles['label']!=3]
di={0:0, 1:1, 2:2, 4:3,6:4,7:5,8:6}
cles['label']=cles['label'].map(di)
print(cles.label.tolist())"""
#eco=cles[cles['label']==0]
#cles=cles[cles['label']!=0]
#eco=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/classification_camembert/economie_cles.csv")
#eco=eco.head(250)
non_cles=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/classification_camembert/keyWords_nonKW.csv")
non_cles=non_cles[non_cles['label']!=5]
non_cles=non_cles[non_cles['label']!=3]
di={0:0, 1:1, 2:2, 4:3,6:4,7:5,8:6}
non_cles['label']=non_cles['label'].map(di)

TOPICS  = ['economie','environnement','immigration_labeled',
           'laicite','sante','sécurité','autre']
           
Full_Data = pd.DataFrame([] ,  columns=['text','label'])

for idx , topic in enumerate(TOPICS) : 
  PATH = os.path.join('/content/drive/MyDrive/Topics/' ,topic +'.csv')
  df = pd.read_csv(PATH)[:1400] 
  df['label'] = idx
  Full_Data = pd.concat([Full_Data,df],axis=0, ignore_index=False)

pdList = [Full_Data, cles, non_cles]
df = pd.concat(pdList, ignore_index=True)
df.text=df.text.astype(str)
print(df)
for i in range(df.shape[0]):
  df.text[i]=clean_data().clean(df.text[i])
print(df)
X_train,X_test,y_train,y_test= train_test_split(df.text.tolist(),df.label.tolist(),shuffle=True,test_size=0.2)


                                                    text label
0      Alors que le projet de loi est examiné lundi à...     0
1      Le 1er juin 2009, le vol Air France AF 447 dis...     0
2      Le projet de loi, qui vise à donner un nouveau...     0
3      L’abondance des récoltes a fait baisser le cou...     0
4      La question de l’avenir de l’Etat-providence d...     0
...                                                  ...   ...
15956  Développé par le studio Westwood à la fin de l...     6
15957  Lors d’une conférence, Fang Binxing a tenté, e...     6
15958  Rentrée littéraire. Chaud New York ! Avec l'éc...     6
15959  Sous la direction de Maurice Godelier, les Edi...     6
15960  Un doublé de son attaquant vedette et une erre...     6

[15961 rows x 2 columns]
                                                    text label
0      alors que le projet de loi est examiné lundi à...     0
1      le  juin  le vol air france af  disparaissait ...     0
2      le projet de loi qui v

In [ ]:
fine_tuning(X_train,X_test,y_train,y_test)

loading configuration file https://huggingface.co/camembert-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f459e43c5ebb871abbf9209195563bff6a11547fd9532047739667c394833221.e23d229c54bcc6f67d337b8b2dd111b0e3dc01fa854bfecd3efdeb8c955749e6
Model config CamembertConfig {
  "_name_or_path": "camembert-base",
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 

Step 1 : Loading Model & Tokenizer 


loading weights file https://huggingface.co/camembert-base/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/7e23f45751ad1fed420ca9f03bb37a279dc98a56c75bf25e671129237e2c893c.ee4d4253e08a7cf9697c0671fd8f022483dbf586691a7b32ead55493a34d72b2
Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly iden

Step 2 : Encoding Tokens


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


...

***** Running Evaluation *****
  Num examples = 3193
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in /content/drive/MyDrive/Colab_Notebooks/classification_camembert/epochs/Epoch_1/config.json
Model weights saved in /content/drive/MyDrive/Colab_Notebooks/classification_camembert/epochs/Epoch_1/pytorch_model.bin
loading configuration file /content/drive/MyDrive/Colab_Notebooks/classification_camembert/epochs/Epoch_1/config.json
Model config CamembertConfig {
  "_name_or_path": "/content/drive/MyDrive/Colab_Notebooks/classification_camembert/epochs/Epoch_1",
  "architectures": [
    "CamembertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": 

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.433925,0.920138


***** Running Evaluation *****
  Num examples = 3193
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in /content/drive/MyDrive/Colab_Notebooks/classification_camembert/epochs/Epoch_2/config.json
Model weights saved in /content/drive/MyDrive/Colab_Notebooks/classification_camembert/epochs/Epoch_2/pytorch_model.bin
loading configuration file /content/drive/MyDrive/Colab_Notebooks/classification_camembert/epochs/Epoch_2/config.json
Model config CamembertConfig {
  "_name_or_path": "/content/drive/MyDrive/Colab_Notebooks/classification_camembert/epochs/Epoch_2",
  "architectures": [
    "CamembertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": 

Epoch,Training Loss,Validation Loss


**Test 1 + 2:** 

In [ ]:
from tqdm import tqdm
def test_overfit_underfit (train_texts,test_texts,train_label,test_label,PATH) : 
  model = AutoModelForSequenceClassification.from_pretrained(PATH)
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  # Test 1 : Underfit ( On Train Dataset ) : 
  train_encoding=tokenizer(train_texts,truncation=True,padding=True,max_length = 20)
  train_dataset=Dataset(train_encoding,train_label)  
  train_loader=DataLoader(train_dataset, batch_size=64, shuffle=True)  
  total = 0 
  train_accuracy = 0  
  with torch.no_grad():
      for batch in (train_loader):
          input_ids=batch['input_ids']
          attention_mask=batch['attention_mask']
          labels=batch['labels']
          outputs=model(input_ids, attention_mask=attention_mask, labels=labels)
          predictions=F.softmax(outputs.logits, dim=1)
          pred= torch.argmax(predictions, dim=1)
          total += labels.size(0)
          train_accuracy += (pred == labels).sum().item()
      train_accuracy = (100 * train_accuracy / total)

  print("train_accuracy:",train_accuracy )

  # Test 2 : Overfit  ( On Test Dataset ) : 
  test_encoding=tokenizer(test_texts,truncation=True,padding=True,max_length = 20)
  test_dataset=Dataset(test_encoding,test_label)  
  test_loader=DataLoader(test_dataset, batch_size=64, shuffle=True)   
  total = 0 
  test_accuracy = 0 
  with torch.no_grad():
      for batch in (test_loader):
          input_ids=batch['input_ids']
          attention_mask=batch['attention_mask']
          labels=batch['labels']
          outputs=model(input_ids, attention_mask=attention_mask, labels=labels)
          predictions=F.softmax(outputs.logits, dim=1)
          """
          pred = []
          for x in predictions : 
            if (x[0].item() >= 0.6 ) : 
              pred.append(0)
            elif (x[2].item() >= 0.6) : 
              pred.append(2)
            else : 
              pred.append(1)
          pred = torch.Tensor(pred)
          """
          pred= torch.argmax(predictions, dim=1)
          total += labels.size(0)
          test_accuracy += (pred == labels).sum().item()
      test_accuracy = (100 * test_accuracy / total)
  print("test_accuracy:",test_accuracy )

In [ ]:
from transformers import  AutoTokenizer, AutoModelForSequenceClassification
PATH = '/content/drive/MyDrive/Colab_Notebooks/classification_camembert/res49/Epoch_5'
model = AutoModelForSequenceClassification.from_pretrained(PATH)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

In [ ]:
test_overfit_underfit (X_train,X_test,y_train,y_test,PATH)

loading configuration file /content/drive/MyDrive/Colab_Notebooks/classification_camembert/epochs/Epoch_1/config.json
Model config CamembertConfig {
  "_name_or_path": "/content/drive/MyDrive/Colab_Notebooks/classification_camembert/epochs/Epoch_1",
  "architectures": [
    "CamembertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden

train_accuracy: 99.625
test_accuracy: 99.75


**Test 3+4**

In [ ]:
def test_contraintes (comments,labels,PATH) : 
  # Passer comments,labels neutre puis comments,labels de Positive + neutre 
  test_encoding=tokenizer(comments,truncation=True,padding=True,max_length = 20)
  test_dataset=Dataset(test_encoding,labels) 
  test_loader=DataLoader(test_dataset, batch_size=64, shuffle=True)  
  total = 0 
  test_accuracy = 0 
  with torch.no_grad():
      for batch in (test_loader):
          input_ids=batch['input_ids']
          attention_mask=batch['attention_mask']
          labels=batch['labels']
          outputs=model(input_ids, attention_mask=attention_mask, labels=labels)
          predictions=F.softmax(outputs.logits, dim=1)
          """
          pred = []
          for x in predictions : 
            if (x[0].item() >= 0.6 ) : 
              pred.append(0)
            elif (x[2].item() >= 0.6) : 
              pred.append(2)
            else : 
              pred.append(1)
          pred = torch.Tensor(pred)
          """
          pred= torch.argmax(predictions, dim=1)
          total += labels.size(0)
          test_accuracy += (pred == labels).sum().item()
      test_accuracy = (100 * test_accuracy / total)

  print("accuracy:",test_accuracy )


In [ ]:
# ATTENTION positive/negative/neutre mta3 train + test  mch 
for i in range(6):
  #df= pd.read_csv("/content/drive/MyDrive/Full_Data.csv")
  df1 = df[df['label'] ==i]
  df1.dropna(subset=['text'],inplace=True)
  comments= (df1.text.tolist())
  labels=df1.label.tolist()
  print("label:" ,i )
  print(test_contraintes (comments,labels,PATH))

label: 0


KeyboardInterrupt: ignored

In [ ]:
# import numpy as np
# test_contraintes (comments,labels,PATH)

**Test 5 :**

In [ ]:
import csv
def test5(comment) : 
  # Passer comments,labels neutre puis comments,labels de Positive + neutre 
  test_encodings=tokenizer([comment], truncation=True, padding=True)
  prediction_inputs = torch.tensor(test_encodings['input_ids'])
  prediction_masks = torch.tensor(test_encodings['attention_mask'])
  with torch.no_grad():
    outputs=model(input_ids=prediction_inputs, attention_mask=prediction_masks)
    predictions=F.softmax(outputs.logits, dim=1)
    print(predictions)
    pred = []
    for x in predictions : 
        index=(x== max(x).item()).nonzero(as_tuple=True)[0].item()
        #print((x== max(x).item()).nonzero(as_tuple=True)[0].item())
        if (index==4 or index==0 or index==1 or index==5) : 
            return index
        elif((index==3 or index==2) and max(x).item()>0.5):
          return index
        else:
          return 6
    #pred = torch.Tensor(pred)"""
    #pred1= torch.argmax(predictions, dim=1)
    #return pred1
#PATH = '/content/drive/MyDrive/Colab_Notebooks/classification_camembert/epochs/Epoch_4'
#classes=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/classification_camembert/8classe_updated.csv")
#with open("/content/drive/MyDrive/Colab_Notebooks/classification_camembert/resultat47.csv", 'w', encoding='UTF8') as f:
                    #writer = csv.writer(f)
                    #writer.writerow(['text', 'label', 'predicted'])
#for i in range(classe.shape[0]):
  #print([classe.label[i], test5(classe.text[i]).item()])


**Test 6**

In [ ]:
# text_sentence = "la france est de dégueulasse "
def test6(comments,labels,PATH) : 
  # Passer comments,labels neutre puis comments,labels de Positive + neutre 
  test_encoding=tokenizer(comments,truncation=True,padding=True,max_length = 15)
  test_dataset=Dataset(test_encoding,labels) 
  test_loader=DataLoader(test_dataset, batch_size=64, shuffle=True)  
  total = 0 
  test_accuracy = 0 
  with torch.no_grad():
      for batch in (test_loader):
          input_ids=batch['input_ids']
          attention_mask=batch['attention_mask']
          labels=batch['labels']
          outputs=model(input_ids, attention_mask=attention_mask, labels=labels)
          predictions=F.softmax(outputs.logits, dim=1)
          pred= torch.argmax(predictions, dim=1)
          total += labels.size(0)
          test_accuracy += (pred == labels).sum().item()
      test_accuracy = (100 * test_accuracy / total)

  print("accuracy:",test_accuracy )

classes=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/classification_camembert/8classe_updated.csv")
t= (classes.text.tolist())
l=classes.label.tolist()
test6(t,l,PATH)

accuracy: 3.1578947368421053


In [ ]:
print(classes)
#t=liste.text.tolist()
#l=liste.label.tolist()
for i in [4,0,2,5,1,3]:
  #df= pd.read_csv("/content/drive/MyDrive/Full_Data.csv")
  #classes=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/classification_camembert/8classe_updated.csv")
  df1= classes[classes['label'] ==i]
  #df1.dropna(subset=['text'],inplace=True)
  t= (df1.text.tolist())
  l=df1.label.tolist()
  print("label:" ,i )
  print(test6(t,l,PATH))

                                                 text  label
0   Organisation mondiale de la Santé (OMS) est l'...      4
1   L’importance de la lutte contre les maladies t...      4
2   utte contre les maladies transmissibles est au...      4
3   il est courant d’affirmer que la charge de mal...      4
4   si les campagnes des autorités sanitaires ne s...      4
..                                                ...    ...
90  Le problème c'est que certaine personnes mal i...      3
91  Être laïc, c'est respecter chaque religion et ...      3
92  Traiter la laïcité de négative quand on interd...      3
93   La loi de 1905 ne comporte pas une fois le te...      3
94  Non mais Fillon et ses propos sur les musulman...      3

[95 rows x 2 columns]
label: 4
accuracy: 0.0
None
label: 0
accuracy: 0.0
None
label: 2
accuracy: 0.0
None
label: 5
accuracy: 0.0
None
label: 1
accuracy: 0.0
None
label: 3
accuracy: 0.0
None


In [ ]:
classe=pd.read_csv("/content/drive/MyDrive/finetune_updated.csv")
for i in range(6):
  df1= classe[classe['label'] ==i]
  t= (df1.text.tolist())
  l=df1.label.tolist()
  print("label:" ,i )
  print(test6(t,l,PATH))

label: 0
accuracy: 61.0
None
label: 1
accuracy: 62.0
None
label: 2
accuracy: 80.0
None
label: 3
accuracy: 62.0
None
label: 4
accuracy: 50.0
None
label: 5
accuracy: 56.0
None


In [ ]:
classes.to_csv("/content/drive/MyDrive/Colab_Notebooks/classification_camembert/8classe_updated.csv", index=False)

In [ ]:
classe.to_csv("/content/drive/MyDrive/finetune_updated.csv", index=False)

In [ ]:
import csv
d=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/filter/data1_filté.csv")
with open("/content/drive/MyDrive/Colab_Notebooks/classification_camembert/resultat49.csv", 'a', encoding='UTF8') as f:
                    #writer = csv.writer(f)
                    for x in d['comment']:
                      print([x, test5(x)])
                      #writer.writerow([x, test5(x)])

tensor([[0.0257, 0.0263, 0.7543, 0.1021, 0.0172, 0.0252, 0.0492]])
['zemmour lui a porté la poisse pauvre nicolas il a perdu tout en  mois et en plus il est mis en examen', 2]
tensor([[0.0392, 0.0305, 0.6627, 0.1233, 0.0245, 0.0630, 0.0567]])
['mr macron à trouvé sont bras droit', 2]
tensor([[0.0306, 0.0290, 0.3278, 0.5278, 0.0244, 0.0265, 0.0339]])
['il a choisi de mettre lepen comme première opposante lors des législatives à la place de la gauche avec son bras droit communiste elle pose un vrai problème de positionnement sur les terres de nupes', 3]
tensor([[0.0281, 0.0275, 0.6598, 0.2031, 0.0193, 0.0243, 0.0379]])
["en fait c'est macron lui-même le vrai premier ministre de la france", 2]
tensor([[0.0302, 0.0291, 0.5065, 0.3454, 0.0223, 0.0236, 0.0428]])
["pauvre macron il est à la recherche d'un  ministre grandeur et décadence on ne créer pas la grandeur d'une nation avec de piètres politiciens", 2]
tensor([[0.6577, 0.0404, 0.0422, 0.0460, 0.0862, 0.0348, 0.0927]])
['pour favoriser 

KeyboardInterrupt: ignored

In [ ]:
cle=cles[cles['label']==3]
cle

,text,label
2000,laïcisation voile sécularité séparation laïcis...,3
2001,musulman sécularité séparatisme religieux loi ...,3
2002,sécularité sécularité laïcité laïque séparatis...,3
2003,religieux religion musulman séparation laïcité...,3
2004,séparation religion islamophobie islamophobie ...,3
...,...,...
2495,sécularité séparation voile religieux signe re...,3
2496,musulman séparatisme religieux religion laïcis...,3
2497,séparatisme religieux islamophobie voile loi d...,3
2498,signe religieux laïcité islamophobie sécularit...,3
